In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [31]:
# from new_loader import load_process_split_doc, load_process_split_doc_law
# import os
# # tax_etc 파일들 정의의
# doc_options = [
#     {
#         "filename":"연말정산_신고안내", 
#         "page_ranges": [(17, 426)],
#         "table_pages_range": "17-425",
#         "replace_string": [
#             {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
#             {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
#             {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
#             {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
#             {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
#             {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
#             {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
#             {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
#             {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
#             {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
#             {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
#             {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
#             {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
#             {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
#             {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
#             {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
#             {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
#             {"target_str": r"I\. 종교인소득이란?\n\d+|"},
#             {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
#             {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
#             {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
#             {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
#             {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
#             {"target_str": r"\bNaN\b"},
#             {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
#             {"target_str": r"\s+", "replace_str": r" "},
#         ]
#     },
#     {
#         "filename":"연말정산_주택자금·월세액_공제의이해", 
#         "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
#         "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
#         "replace_string": [
#             {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
#             {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
#             {"target_str": r"\s+", "replace_str": " " },
#         ]
#     },
#     {
#         "filename":"2024_핵심_개정세법", 
#         "page_ranges": [(19, 44), (47, 71), (75, 161)],
#         "table_pages_range": "19-44,47-71,75-161",
#         "replace_string": [
#             {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
#                 r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
#                 r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
#                 r"\s*3\s*❚국민･기업\s*납세자용\s*|"
#                 r"\s*2\s*0\s*2\s*4\s|"
#                 r"\s한국세무사회\s|" 
#                 r"\n7\n❚국민･기업 납세자용|"
#                 r"\n71\n❚상세본|"),  
#             },
#             {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
#             {"target_str": r"\s+", "replace_str": " " },
#         ]
#     },
#     {
#         "filename":"주요_공제_항목별_계산사례", 
#         "page_ranges": [(1,1), (4, 21)],
#         "table_pages_range": "1,4-21",
#         "replace_string": [
#             {"target_str": r"\bNaN\b"},
#             {"target_str": r"\s+", "replace_str": " " },
#         ]
#     }
    
# ]


# # vector store 에 넣을 데이터 document 모으기 -> all_documents
# # all_documents = []

# # folders = ["data/tax_law1", "data/tax_law2", "data/tax_law3", "data/tax_law4", "data/tax_law5", "data/tax_law6"]
# # # folders = ["data/tax_law"]
# # # load - 세법 
# # for folder in folders:
# #     law_files = get_file_names(folder)
# #     for file in law_files:
# #         all_documents.extend(load_process_split_doc_law(file, directory_path=folder))

# # # load - 참고자료
# # for doc in doc_options:
# #     all_documents.extend(load_process_split_doc(**doc))

In [2]:
from new_loader import load_process_split_doc_law
from langchain.schema import Document
from langchain.vectorstores import Chroma
import os
from langchain.embeddings.openai import OpenAIEmbeddings

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
)

folders = ["data/tax_law1", "data/tax_law2", "data/tax_law3", "data/tax_law4", "data/tax_law5", "data/tax_law6"]

for folder in folders:
    law_files = get_file_names(folder)
    for file in law_files:
        documents=load_process_split_doc_law(file, directory_path=folder)
        vector_store.add_documents(documents)

vector_store.persist()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_21428\2707382520.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_21428\2707382520.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(
C:\Users\Playdata\AppData\Local\Temp\ipykernel_21428\2707382520.py:2

In [3]:
# Retreiver 생성
retriever = vector_store.as_retriever(
    # search_type="mmr",
    search_kwargs={"k": 5}
)

In [5]:
retriever.invoke("개별소비세법 제 2조는?")

[Document(metadata={'filename': '개별소비세법_시행규칙', 'source': '개별소비세법_시행규칙.pdf'}, page_content='개별소비세법 시행규칙 기획재정부 (환경에너지세제과) 044-215-4331, 4336 기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336 제1조(용도별 탄력세율을 적용하는 물품의 범위 등) ① 「개별소비세법 시행령」(이하 “영”이라 한다) 제2조의2제1항제 2호나목에서 “기획재정부령으로 정하는 가정용ㆍ상업용 물품”이란 다음 각 호의 어느 하나에 해당하는 자에게 공급 하는 물품을 말한다. 1. 「액화석유가스의 안전관리 및 사업법」 제2조제5호에 따른 액화석유가스 충전사업자 2. 「도시가스사업법」 제2조제4호에 따른 일반도시가스사업을 하는 자로서 같은 법 시행령 제1조의2제2호가목에 따른 석유가스를 공급하는 자 3. 「고압가스 안전관리법」에 따른 고압가스 제조(액화석유가스 제조만 해당한다)허가를 받은 자. 다만, 공급받은 물 품을 「액화석유가스의 안전관리 및 사업법」 제2조제5호에 따른 액화석유가스 충전사업자에게 재공급하는 경우 에 한정한다. ② 「개별소비세법」(이하 “법”이라 한다) 제1조제2항제4호사목 단서 및 영 제2조의2제1항제4호나목에서 “기획재정 부령으로 정하는 것”이란 다음 각 호의 물품을 말한다. 1. 「도시가스사업법」 제2조제8호에 따른 천연가스수출입업자(이하 이 항에서 “천연가스수출입업자”라 한다)가 수 입하는 천연가스 또는 국내에서 제조하는 천연가스로서 다음 각 목의 어느 하나에 해당하는 자에게 공급하는 물 품 가. 「도시가스사업법」 제2조제2호에 따른 가스도매사업자(영 제2조의2제1항제4호가목 및 다목의 물품 또는 「도 시가스사업법 시행규칙」 제2조제2항제2호가목의 물품을 공급하기 위한 경우는 제외한다) 나. 「도시가스사업법」 제2조제2호에 따른 일반도시가스사업자 및 도시가스충전사업자(영 제2조의2제1항제4호가 목 및 다목의 물품을 공급하기 위한 경우는 

In [6]:
from langchain.prompts import ChatPromptTemplate
# Prompt Template 생성
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 
    문서에 없는 내용일 경우 모른다고 대답해주세요.
    
    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
    2. **관련 세법 조항 검색**:
        - 질문의 핵심 키워드와 가장 관련 있는 세법 조항이나 시행령을 우선적으로 찾습니다.
        - 필요한 경우, 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높입니다.
    3. **질문 유형 판단**:
        - **정의 질문**: 특정 용어나 제도의 정의를 묻는 경우.
        - **절차 질문**: 특정 제도의 적용이나 신고 방법을 묻는 경우.
        - **사례 질문**: 구체적인 상황에 대한 세법 해석을 요청하는 경우.
    4. **답변 생성**:
        - 법률 조항에관한 질문이라면 그 조항에 관한 전체 내용을 가져온 후 요약 정리하여 이해하게 설명한다.
        - 질문 유형에 따라 관련 정보를 구조적으로 작성하며, 중요 세법 조문과 요약된 내용을 포함합니다.
        - 비전문가도 이해할 수 있도록 용어를 친절히 설명합니다.

    **답변 작성 가이드라인**:
    - **간결성**: 답변은 간단하고 명확하게 작성하되, 법 조항에 관한 질문일 경우 관련 법 조문의 전문을 명시합니다.
    - **구조화된 정보 제공**:
        - 세법 조항 번호, 세법 조항의 정의, 시행령, 관련 규정을 구체적으로 명시합니다.
        - 복잡한 개념은 예시를 들어 설명하거나, 단계적으로 안내합니다.
    - **신뢰성 강조**:
        - 답변이 법적 조언이 아니라 정보 제공 목적임을 명확히 알립니다.
        - "이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."를 추가합니다.
    - **정확성**:
        - 법령 및 법률에 관한질문은 추가적인 내용없이 한가지 content에 집중하여 답변한다.
        - 법조항에대한 질문은 시행령이나 시행규칙보단 해당법에서 가져오는것에 집중한다.
      
    **추가적인 사용자 지원**:
    - 답변 후 사용자에게 주제와 관련된 후속 질문 두 가지를 제안합니다.
    - 후속 질문은 사용자가 더 깊이 탐구할 수 있도록 설계한다.

    **예외 상황 처리**:
    - 사용자가 질문을 모호하게 작성한 경우:
        - "질문이 명확하지 않습니다. 구체적으로 어떤 부분을 알고 싶으신지 말씀해 주시겠어요?"와 같은 문구로 추가 정보를 요청합니다.
    - 질문이 세법과 직접 관련이 없는 경우:
        - "이 질문은 제가 학습한 대한민국 세법 범위를 벗어납니다."라고 알리고, 세법과 관련된 새로운 질문을 유도합니다.

    **추가 지침**:
    - 개행문자 두 개 이상은 절대 사용하지 마세요.
    - 질문 및 답변에서 사용된 세법 조문은 최신 데이터에 기반해야 합니다.
    - 질문이 복합적인 경우, 각 하위 질문에 대해 별도로 답변하거나, 사용자에게 우선순위를 확인합니다.

    **예시 답변 템플릿**:
    - "질문에 대한 답변: ..."
    - "관련 세법 조항: ..."
    - "추가 설명: ..."
    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [9]:
# print(chain.invoke("법인세법 제1조 에 대해 알려줘"))
# retriever.invoke("법인세법 제1조 에 대해 알려줘")
print(chain.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘"))
retriever.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘")
# print(chain.invoke("개별소비세법 제1조 에 대해 알려줘"))
# retriever.invoke("개별소비세법 제1조 에 대해 알려줘")

질문에 대한 답변: 개별소비세법 제1조는 개별소비세의 과세대상과 세율에 대해 규정하고 있습니다.

관련 세법 조항: 
1. 개별소비세는 특정한 물품, 특정한 장소 입장행위, 특정한 장소에서의 유흥음식행위 및 특정한 장소에서의 영업행위에 대하여 부과됩니다.
2. 과세물품과 그 세율은 다음과 같습니다:
   - 투전기, 오락용 사행기구, 그 밖의 오락용품: 물품가격에 20% 세율
   - 수렵용 총포류: 물품가격에 20% 세율
   - 보석, 귀금속 제품, 고급 시계 등: 기준가격 초과 부분에 20% 세율
   - 배기량이 2,000cc를 초과하는 승용자동차: 물품가격에 5% 세율
   - 휘발유: 리터당 475원 
   - 경유: 리터당 340원
   - 등유: 리터당 90원
   - 중유: 리터당 17원
   - 담배: 종류별 세율은 별표에 따름

추가 설명: 개별소비세는 특정 물품과 행위에 대해 부과되는 세금으로, 주로 사치품이나 특정 유흥 행위에 적용됩니다. 이를 통해 소비 억제와 세수 확보를 동시에 추구합니다.

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다. 

추가로 궁금하신 점은 다음 중에서 선택하실 수 있습니다:
1. 개별소비세의 신고 절차에 대해 알고 싶으신가요?
2. 개별소비세의 면세 대상에 대해 알아보시겠어요?


[Document(metadata={'filename': '개별소비세법', 'source': '개별소비세법.pdf'}, page_content='개별소비세법 기획재정부 (환경에너지세제과) 044-215-4331, 4336 기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336 제1조(과세대상과 세율) ① 개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행 위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다. ② 개별소비세를 부과할 물품(이하 “과세물품”이라 한다)과 그 세율은 다음과 같다. 1. 다음 각 목의 물품에 대해서는 그 물품가격에 100분의 20의 세율을 적용한다. 가. 투전기(投錢機), 오락용 사행기구(射倖器具), 그 밖의 오락용품 나. 수렵용 총포류 2. 다음 각 목의 물품에 대해서는 그 물품가격 중 대통령령으로 정하는 기준가격(이하 “기준가격”이라 한다)을 초과 하는 부분의 가격(이하 이 호에서 “과세가격”이라 한다)에 해당 세율을 적용한다. 가. 다음의 물품에 대해서는 과세가격의 100분의 20 1) 보석, 진주, 별갑(鼈甲), 산호, 호 박(琥珀) 및 상아와 이를 사용한 제품(나석을 사용한 제품은 포함한다) 2) 귀금속 제품 3) 삭제 4) 고급 시계 5) 고급 융단 6) 고급 가방 나. 다음의 물품에 대해서는 과세가격의 100분의 20 1) 고급 모피와 그 제품 2) 고급 가구 3. 다음 각 목의 자동차에 대해서는 그 물품가격에 해당 세율을 적용한다. 가. 배기량이 2천시시를 초과하는 승용자동차와 캠핑용자동차: 100분의 5 나. 배기량이 2천시시 이하인 승용자동차(배기량이 1천시시 이하인 것으로서 대통령령으로 정하는 규격의 것은 제외한다)와 이륜자동차 : 100분의 5 다. 전기승용자동차(「자동차관리법」 제3조제2항에 따른 세부기준을 고려하여 대통령령으로 정하는 규격의 것은 제외한다): 100분의 5 4. 다음 각 목의 물품에 대해서는 그 수량에 해당 세율을 적용

In [10]:
print(chain.invoke("연봉 4000에 회사를 다니는 근로자인데 개인사업도 하고있어. 개인사업으로는 올해 3000만원의 소득이 이루어졌는데 연말정산을 어떻게 해야할까?"))
retriever.invoke("연봉 4000에 회사를 다니는 근로자인데 개인사업도 하고있어. 개인사업으로는 올해 3000만원의 소득이 이루어졌는데 연말정산을 어떻게 해야할까?")

연말정산은 주로 근로소득에 대한 세금 정산을 의미합니다. 하지만, 귀하의 경우에는 근로소득 외에 개인사업 소득이 있으므로, 종합소득세 신고를 통해 두 소득을 모두 신고해야 합니다. 다음은 기본적인 절차입니다:

1. **근로소득 연말정산**:
   - 연말정산은 근로소득에 대해 회사가 연말에 소득세를 정산하는 절차입니다. 회사는 귀하의 연봉 4,000만원에 대해 이미 원천징수한 세금을 바탕으로 연말정산을 진행하게 됩니다.
   - 연말정산 시, 각종 공제 항목(예: 인적공제, 보험료 공제, 의료비 공제 등)을 적용받아 최종 세액을 산출하게 됩니다.

2. **종합소득세 신고**:
   - 개인사업 소득인 3,000만원은 종합소득세 신고 대상입니다. 5월 1일부터 5월 31일까지 국세청 홈택스 등을 통해 신고하셔야 합니다.
   - 이때, 근로소득과 사업소득을 합산하여 종합소득세를 계산합니다.
   - 종합소득세 신고 시, 사업소득에 대해 추가로 적용받을 수 있는 경비나 공제 항목을 고려하여 신고하시면 됩니다.

3. **세액 계산 및 납부**:
   - 두 소득을 합산한 총 소득에 대해 종합소득세율을 적용하여 세액을 계산합니다.
   - 이미 원천징수된 근로소득세와 비교하여 추가로 납부해야 할 세액이나 환급받을 세액을 확인합니다.

4. **추가 공제 및 절세 전략**:
   - 사업소득과 관련하여 실제로 지출한 경비나 세액공제 항목을 최대한 활용하여 세액을 줄일 수 있습니다.
   - 필요시 세무 전문가와 상담하여 절세 전략을 세우는 것도 고려할 수 있습니다.

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다. 더 궁금한 점이 있으면 말씀해 주세요.


[Document(metadata={'filename': '법인세법', 'source': '법인세법.pdf'}, page_content='적용할 때 부동산임대업을 주된 사업으로 하는 등 대통령령으로 정하는 요건에 해당하는 내국법인의 경 우에는 같은 항 각 호의 금액의 합계액의 100분의 50을 초과하는 금액은 해당 사업연도의 소득금액을 계산할 때 손 금에 산입하지 아니한다. ⑥ 기업업무추진비의 범위 및 가액의 계산, 지출증명 보관 등에 필요한 사항은 대통령령으로 정한다. 제26조(과다경비 등의 손금불산입) 다음 각 호의 손비 중 대통령령으로 정하는 바에 따라 과다하거나 부당하다고 인정 하는 금액은 내국법인의 각 사업연도의 소득금액을 계산할 때 손금에 산입하지 아니한다. 1. 인건비 2. 복리후생비 3. 여비(旅費) 및 교육ㆍ훈련비 4. 법인이 그 법인 외의 자와 동일한 조직 또는 사업 등을 공동으로 운영하거나 경영함에 따라 발생되거나 지출된 손비 5. 제1호부터 제4호까지에 규정된 것 외에 법인의 업무와 직접 관련이 적다고 인정되는 경비로서 대통령령으로 정 하는 것 제27조(업무와 관련 없는 비용의 손금불산입) 내국법인이 지출한 비용 중 다음 각 호의 금액은 각 사업연도의 소득금액 을 계산할 때 손금에 산입하지 아니한다. 1. 해당 법인의 업무와 직접 관련이 없다고 인정되는 자산으로서 대통령령으로 정하는 자산을 취득ㆍ관리함으로써 생기는 비용 등 대통령령으로 정하는 금액 2. 제1호 외에 해당 법인의 업무와 직접 관련이 없다고 인정되는 지출금액으로서 대통령령으로 정하는 금액 제27조의2(업무용승용차 관련비용의 손금불산입 등 특례) ① 「개별소비세법」 제1조제2항제3호에 해당하는 승용자동 차(운수업, 자동차판매업 등에서 사업에 직접 사용하는 승용자동차로서 대통령령으로 정하는 것과 연구개발을 목적 으로 사용하는 승용자동차로서 대통령령으로 정하는 것은 제외하며, 이하 이 조 및 제74조의2에서 “업무용승용차”라 한다)에 대한 감가상각비는 각 사업연도의 소득금액을 계산

In [11]:
print(chain.invoke("현재 농사를 법인회사 형태로 소득을 벌어들이고있는데 세금혜택이있을까?"))
retriever.invoke("현재 농사를 법인회사 형태로 소득을 벌어들이고있는데 세금혜택이있을까?")

농업 분야에서 법인 형태로 소득을 벌어들이는 경우, 대한민국의 세법에서는 다양한 세금 혜택을 제공하고 있습니다. 특히, 조세특례제한법에서는 농업 관련 법인에 대한 세제 지원을 규정하고 있습니다. 다음은 몇 가지 주요 혜택입니다:

1. **법인세 감면**: 농업 법인은 일정 요건을 충족할 경우 법인세를 감면받을 수 있습니다. 이는 주로 농업 생산이나 가공, 유통을 목적으로 하는 법인에 적용됩니다.

2. **소득세 감면**: 농업 법인이 농작물을 직접 생산하여 판매하는 경우, 일정 부분의 소득에 대해 소득세를 감면받을 수 있습니다.

3. **부가가치세 면제**: 직접 생산한 농산물의 판매에 대해서는 부가가치세가 면제될 수 있습니다.

4. **기타 세제 혜택**: 농업 법인은 특정 기계나 설비에 대한 투자에 대해 세액 공제나 감면을 받을 수 있으며, 농업 관련 연구 개발에 대한 세제 지원도 받을 수 있습니다.

세제 혜택을 받기 위해서는 각 혜택별로 정해진 요건을 충족해야 하며, 해당 혜택이 언제까지 유효한지는 법령에 따라 달라질 수 있습니다. 구체적인 혜택을 받기 위해서는 전문 세무사나 회계사의 자문을 받는 것이 좋습니다.

"이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."

추가적으로 궁금하신 점이 있다면, "농업 법인이 받을 수 있는 구체적인 세액 공제는 어떤 것이 있나요?" 또는 "농업 법인 설립 시 주의해야 할 세법 요건은 무엇인가요?"와 같은 질문을 고려하실 수 있습니다.


[Document(metadata={'filename': '조세특례제한법', 'source': '조세특례제한법.pdf'}, page_content='말한다)은 해당 사업연도의 소득금액을 계 산할 때 손금에 산입한다. ③ 제1항에 따라 법인이 주주등으로부터 주식등을 무상으로 받아 소각함으로써 해당 법인의 다른 주주등이 얻는 이 익에 대해서는 「상속세 및 증여세법」에 따른 증여 또는 「법인세법」에 따른 익금으로 보지 아니한다. 다만, 주식등 을 증여한 주주등의 특수관계인에 대해서는 그러하지 아니하다. ④ 제1항부터 제3항까지의 규정을 적용할 때 재무구조개선계획의 내용 및 승인기준, 특수관계인의 범위, 세액 감면 의 신청, 그 밖에 필요한 사항은 대통령령으로 정한다. 제45조의2(공공기관의 구조개편을 위한 분할에 대한 과세특례) 「공공기관의 운영에 관한 법률」 제4조에 따라 공공기 관으로 지정된 내국법인(이하 “공공기관”이라 한다)이 민영화 등의 구조개편을 위하여 2010년 12월 31일까지 「상법 」 제530조의2부터 제530조의11까지의 규정에 따라 대통령령으로 정하는 분할을 하는 경우로서 그 분할이 대통령령 으로 정하는 요건을 갖춘 경우에는 「법인세법」 제46조제1항 각 호의 요건을 갖춘 분할로 보아 이 법과 「법인세법」 및 「부가가치세법」의 분할에 관한 규정을 적용한다. 제46조(기업 간 주식등의 교환에 대한 과세특례) ① 내국법인(이하 이 조에서 “교환대상법인”이라 한다)의 지배주주ㆍ 출자자 및 그 특수관계인(이하 이 조에서 “지배주주등”이라 한다)이 2017년 12월 31일 이전에 대통령령으로 정하는 재무구조개선계획(대통령령으로 정하는 자가 승인한 것에 한정한다. 이하 이 조에서 “재무구조개선계획”이라 한다 )에 따라 그 소유 주식 또는 출자지분(이하 이 조에서 “주식등”이라 한다) 전부를 양도하고 교환대상법인의 대통령령 으로 정하는 특수관계인이 아닌 다른 내국법인(이하 이 조에서 “교환양수법인”이라 한다)의 주식등을 다음 각 호의 어느 하나에 해당하는 